PyTorch提供两个重要特性：
- n维张量
- 自动求导
我们将使用一个利用三次多项式来拟合y=sin(x) 的问题作为例子。这个网络有4个参数，我们通过梯度下降法来最小化网络的输出和真实输出之间的欧氏距离，以此拟合随机输入的数据。

一、张量
- 热身：Numpy
在介绍 PyTorch 之前，我们先使用 Numpy 来实现一个网络。
Numpy 提供了一个 n 维数组对象和许多操作这些数组的函数。Numpy 是一个用于科学计算的通用框架; 它不包含任何关于计算图、深度学习或梯度的内容。
但是，我们可以很容易地使用 numpy 将一个三阶多项式拟合成正弦函数，方法是通过 numpy 的一系列 operation 手动实现网络的向前和向后传递:

In [17]:
import numpy as np
import math

#创建随机的输入和输出数据
x = np.linspace(-math.pi, math.pi, 2000)  #在指定的间隔内返回均匀间隔的数字。[起始，结束，数量]
# y = np.sin(x)
y = 1 + 2 * x + 3 * x ** 2 + 4 * x ** 3

#随机生成权重
a = np.random.randn()  #randn函数返回一个或一组样本，具有标准正态分布。
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

leanring_rate = 1e-6
for t in range(10000):
    #向前传播：计算预测的 y
    #y = a + bx + cx^2 + dx^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3
    
    #计算并打印 loss
    loss = np.square(y_pred - y).sum()  #求平方和
    if t % 1000 == 999:
        print(t, loss)
        
    #反向传播来计算 a,b,c,d 关于 loss 的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()
    
    #更新权重
    a -= leanring_rate * grad_a
    b -= leanring_rate * grad_b
    c -= leanring_rate * grad_c
    d -= leanring_rate * grad_d
    
print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

999 31.18343170869768
1999 0.7115274728590752
2999 0.018084200831835002
3999 0.0005004161569510489
4999 1.465502851865267e-05
5999 4.438764665261046e-07
6999 1.3696837177327227e-08
7999 4.268448416317019e-10
8999 1.337055270718228e-11
9999 4.1992704514163467e-13
Result: y = 0.9999999783399893 + 1.99999999894079 x + 3.0000000037367123 x^2 + 4.0000000001506635 x^3


- PyTorch：张量
Numpy 框架很棒，但不能利用GPU加速。
PyTorch 张量在概念上与 Numpy 数组类似：张量也是一个 n 维数组，PyTorch 提供了许多函数来操作这些张量。在幕后，张量可以跟踪计算图和梯度。
跟 Numpy 不同的是，PyTorch 的张量可以用GPU来加速计算，只需指定设备即可。
在这里，我们用 PyTorch 张量通过三阶多项式拟合正弦函数。同上一部分一样，我们来手动实现网络中的向前和向后传递。

In [44]:
import torch
import math


dtype = torch.float
# device = torch.device("cpu")
defvice = torch.device("cuda")

# 创建随机输入和输出数据
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
# y = torch.sin(x)
y = 1 + 2 * x + 3 * x ** 2 + 4 * x ** 3

# 随机初始化权重
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(10000):
    # 向前传播：预测 y
    y_pred = a + b * x + c * x ** 2 + d * x **3
    
    # 计算并打印 loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 1000 == 999:
        print(t, loss)
    # 向后传播并计算 a，b，c，d关于 loss 的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()
    
    # 用梯度下降更新权重
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d
    
print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

999 144.64390563964844
1999 2.4527010917663574
2999 0.04366637021303177
3999 0.0008387069683521986
4999 1.9538689230103046e-05
5999 2.5563788312865654e-06
6999 2.5563788312865654e-06
7999 2.5563788312865654e-06
8999 2.5563788312865654e-06
9999 2.5563788312865654e-06
Result: y = 0.9999729990959167 + 1.999958872795105 x + 3.0000059604644775 x^2 + 4.000006198883057 x^3


二、自动求导
- PyTorch：张量和自动求导
上面的例子很简单，手动实现很容易，大型网络就不方便手动实现了。
但有了 PyTorch，我们可以使用 PyTorch 中的 autograd 包，来自动计算神经网络中的反向传递。
使用 autograd 时，网络的向前传递将定义一个计算图; 图中的节点将是张量，边将是从输入张量产生输出张量的函数。
反向传播通过这个图来计算梯度。  

听起来复杂实践起来简单。如果 x 是一个 `x.require _ grad = True` 的张量，那么x 对某个标量值的梯度则可以通过 `x.grad` 获得。  

现在我们通过 PyTorch 张量的自动求导来将三次多项式拟合正弦函数。

In [61]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# defvice = torch.device("cuda:0")

# 创建随机输入和输出数据
# 张量的 requires_grad 默认为 False
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
# y = torch.sin(x)
y = 1 + 2 * x + 3 * x ** 2 + 4 * x ** 3

# 创建四个权重张量 a, b, c, d: y = a + b x + c x^2 + d x^3
# requires_grad=True 意味着我们要计算其梯度
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # 向前传播：通过操作张量来预测 y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # 通过操作张量计算并打印 loss
    # 现在 loss 是一个形状为 （1，）的张量
    # loss.item() 用来获取 loss 的标量值
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    # 通过自动求导来计算反向传播的梯度的过程，PyTorch 会计算所有 requires_grad=True 的标量
    # 自动求导后，a.grad，b.grad等等会作为张量各自保存着 a，b 对应的关于 loss 的梯度
    loss.backward()

    # 手动更新权重（a，b，c，d）的时候，不需要自动求导的计算图追踪他们的计算过程
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # 权重更新后手动清零梯度
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None
            
print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 13642.4794921875
199 9031.787109375
299 5979.84912109375
399 3959.5361328125
499 2622.03955078125
599 1736.507568359375
699 1150.166748046875
799 761.8931274414062
899 504.7524719238281
999 334.44122314453125
1099 221.62559509277344
1199 146.88694763183594
1299 97.36748504638672
1399 64.55303192138672
1499 42.80545425415039
1599 28.389713287353516
1699 18.83257484436035
1799 12.495615005493164
1899 8.292737007141113
1999 5.504903793334961
Result: y = 0.9809680581092834 + 1.9300259351730347 x + 3.003283739089966 x^2 + 4.009953498840332 x^3


- PyTorch：定义新的自动求导公式

实际上，每个基本的 autograd 操作符实际上是两个对 Tensors 进行操作的函数。**正向函数**从输入张量计算输出张量。**反向函数**接收输出张量相对于某个标量值的梯度，并计算输入张量相对于同一标量值的梯度。


在 PyTorch 中，我们可以通过定义 `torch.autograd.Function` 的子类来定义自己的 autograd 操作符,实现前向后向传播。然后，我们可以通过构造一个实例并像调用函数一样调用它，传递包含输入数据的 Tensors 来使用新的 autograd 操作符。

在下面的例子中，我们把模型定义为 $y=a+bP_3(c+dx)$，其中 $P_3(x)=\frac{1}{2}5x^3-3x$ 是三次勒让德多项式。
接下来自定义 autograd 函数来计算 $P_3$ 的前向和后向传播，并用其实现我们夫人模型。

In [70]:
import torch
import math

class LegendrePolynomial3(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        """
        在向前传播当中，我们接收一个 Tensor 作为输入，返回一个 Tensor 作为输出。
        ctx 是一个用来隐藏反向传播信息的上下文对象。
        可以使用 ctx.save_for_backward 方法缓存任意对象，以便在反向传播中使用。
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)
    @staticmethod
    def backward(ctx, grad_output):
        """
        在反向传播中，我们接收一个包含关于 loss 的损失梯度的张量，我们需要计算 loss 关于 input 的梯度。
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)
    
dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")

# 创建随机输入和输出数据
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # 用 Function.apply 方法来使用我们的函数，命名为 P3
    P3 = LegendrePolynomial3.apply
    
    # 我们用自定义的自动求导操作来计算P3
    y_pred = a + b * P3(c + d * x)
    
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
        
    loss.backward()
    
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')
    

99 16.171911239624023
199 13.80992317199707
299 12.206977844238281
399 11.118906021118164
499 10.380213737487793
599 9.878663063049316
699 9.538034439086914
799 9.306766510009766
899 9.149720191955566
999 9.043050765991211
1099 8.970601081848145
1199 8.92138671875
1299 8.88795280456543
1399 8.865248680114746
1499 8.84982681274414
1599 8.83935546875
1699 8.832240104675293
1799 8.82740592956543
1899 8.824119567871094
1999 8.821889877319336
Result: y = -7.463839324373112e-09 + -2.2291290760040283 * P3(2.462566817129641e-09 + 0.2556264400482178 x)


三、nn 模块

- PyTorch：nn

计算图和自动梯度是定义复杂运算符和自动求导的一个非常强大的范例，但是对于大型神经网络来说，光是自动求导还是有点不太够用。

在构建神经网络时，我们经常考虑将计算分层，某几层有可学习的参数，可以在学习过程中进行优化。

在 PyTorch 中，`nn` 包定义了一组模块，它们大致等价于神经网络层。模块的输入和输出虽然都是张量，但是也可以保存内部状态，比如包含了可学习参数的张量。`nn` 包还定义了一组常用于训练神经网络的损失函数。

在下面的例子中，我们使用 `nn` 来实现多项式网络模型：

In [100]:
import torch
import math

x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# 在这个例子中，输出：y 是（x, x^2, x^3）的线性方程，所以我们把它看作一个线性网络
# 准备张量（x, x^2, x^3）
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# 上面代码中，x.unsqueeze(-1) 的形状是（2000，1），p的形状是（3，）
# 这种情况下，广播语义可以获得一个（2000，3）尺寸的张量
# 使用 nn 包可以将我们的模型定义为一系列的层，nn.Sequential 包含其他模块，按顺序执行得到 output
# 线性模块使用线性函数计算 output ，并保存内部张量的权重和偏差。
# Flatten 层将线性层的输出压平为一维张量来匹配 y 的形状

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# nn 还包含常用损失函数的定义，在这个例子中，我们使用均方误差（MSE）作为损失函数
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):
    y_pred = model(xx)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    # 在运行向后传播之前将梯度调零
    model.zero_grad()
    
    # 反向传播: 计算模型所有可学习参数（requires_grad=True）的损失梯度
    loss.backward()
    
    # 使用梯度下降法更新权重。每个参数都是一个张量，所以我们可以像往常一样访问它的梯度
    with torch.no_grad():
        for param in model.parameters():
            param -= leanring_rate * param.grad
    
    # 你可以像访问列表的第一项一样访问模型的第一层
    linear_layer = model[0]
    # 对于线性层，其参数以“权重”和“偏置”的形式存储
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 2744.318603515625
199 1848.897216796875
299 1246.8074951171875
399 841.5919799804688
499 568.632568359375
599 384.59356689453125
699 260.3876953125
799 176.4799346923828
899 119.73834991455078
999 81.33010864257812
1099 55.302528381347656
1199 37.64710235595703
1299 25.656755447387695
1399 17.50545883178711
1499 11.95757007598877
1599 8.177275657653809
1699 5.598736763000488
1799 3.8375887870788574
1899 2.633410930633545
1999 1.8091998100280762
Result: y = 0.9659658074378967 + 1.9729329347610474 x + 3.005871295928955 x^2 + 4.003849983215332 x^3


- PyTorch：优化
目前为止，我们已经通过 `torch.no_grad()` 手动更新了模型的权重。对于 *像随机梯度下降* 这样的简单优化算法来说，还不算难，但是在实践中，我们经常使用更复杂的优化器来训练神经网络，比如 AdaGrad，RMSProp，Adam 等。

PyTorch 中的 `optim` 包抽象了优化算法的思想，并提供了常用优化算法的实现。

在下面的例子中，我们将像以前一样使用 nn 包来定义我们的模型，但是将使用 optim 包提供的 RMSprop 算法来优化模型:

In [17]:
import torch
import math

x = torch.linspace(-math.pi, math.pi, 2000)
# y = torch.sin(x)
y = 1 + 2 * x + 3 * x ** 2 + 4 * x ** 3

# 准备张量（x, x^2, x^3）
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# 使用 nn 来定义模型和损失函数
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# 使用 optim 定义一个优化器来为我们更新模型的权重
# 这里我们将使用 RMSprop
# optim 包含许多其他优化算法
# RMSprop 构造函数的第一个参数用来告诉优化器它应该更新哪些张量
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for t in range(2000):
    y_pred = model(xx)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    # 在向后传递之前，使用优化器对象将它要更新的变量的所有的梯度归零(这些变量是模型的可学习的权重)
    # 这是因为在默认情况下，当调用 backward（）时，梯度是在缓冲区累加的(非覆盖)
    # 查看 torch.autograd.backwards 文档可以了解更多细节
    optimizer.zero_grad()
    
    # 反向传播: 计算 loss 关于模型参数的梯度
    loss.backward()
    
    # 在优化器上调用 step() 函数来更新其参数
    optimizer.step()
    
linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 6221316.5
199 5868048.0
299 5555701.5
399 5261233.5
499 4978179.5
599 4704368.5
699 4439034.0
799 4181887.25
899 3932820.25
999 3691787.75
1099 3458767.5
1199 3233744.0
1299 3016703.75
1399 2807635.0
1499 2606526.0
1599 2413360.5
1699 2228125.5
1799 2050801.875
1899 1881374.0
1999 1719820.125
Result: y = 2.5622239112854004 + 1.9125585556030273 x + 1.5245548486709595 x^2 + 1.5586963891983032 x^3


- PyTorch：自定义 nn 模块

有时候需要更复杂的模型，可以通过子类化`nn.Module`和一个`forward`来定义自己的模块。
在接下来的例子中，我们将三阶多项式作为一个自定义的 Module 子类来实现:

In [38]:
import torch
import math

class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        在构造函数中，我们实例化四个参数，并将它们作为成员参数分配
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        
    def forward(self, x):
        """
        在前向函数中，我们接受一个输入数据的张量，并且必须返回一个用来输出数据的张量。
        我们可以使用构造函数中定义的模块，也可以使用 Tensors 上的任意操作符。
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
    
    def string(self):
        """
        就像 Python 中的任何类一样，您也可以在 PyTorch 模块上自定义方法
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'
    
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 通过实例化上面定义的类来构造我们的模型
model = Polynomial3()

# 构造我们的损失函数和优化器
# SGD 构造函数中 model.parameters ()指的是 nn.Linear 模块中的可学习参数
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    y_pred = model(x)
    
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'Result: {model.string()}')

99 2486.525634765625
199 1681.998046875
299 1139.831787109375
399 774.1256713867188
499 527.2081298828125
599 360.3292236328125
699 247.4293670654297
799 170.96926879882812
899 119.13241577148438
999 83.95153045654297
1099 60.04827117919922
1199 43.78956604003906
1299 32.71812438964844
1399 25.170413970947266
1499 20.019065856933594
1599 16.499235153198242
1699 14.091410636901855
1799 12.4423828125
1899 11.311720848083496
1999 10.535612106323242
Result: y = -0.03431374952197075 + 0.83162522315979 x + 0.005919694900512695 x^2 + -0.08975791186094284 x^3


- PyTorch：控制流+权重共享

作为动态图和权重分配的一个例子，我们要实现一个非常奇怪的模型: 一个三阶-五阶多项式，每次向前传递选择一个介于3到5之间的随机数作为阶数，重复使用相同的权重计算四阶和五阶

对于这个模型，我们可以使用普通的 Python 流控制来实现循环，而且我们可以通过在定义向前传递时多次重用同一个参数来实现权重共享。

我们可以很容易地将这个模型作为一个 Module 子类来实现:

In [45]:
import random
import torch
import math


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        在构造函数中，我们实例化五个参数，并将它们分配为成员x
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))
        
    def forward(self, x):
        """
        对于模型的前向传递，我们随机选择4,5和重用参数 e 来计算这些阶的贡献。
        
        由于每次向前传递都构建一个动态计算图，因此在定义模型向前传递时，我们可以使用常规的 Python 控制流运算符，如循环或条件语句。
        
        这里我们还看到，在定义计算图时，多次重用相同的参数是非常安全的。
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y
    
    def string(self):
        """
        就像 Python 中的任何类一样，您也可以在 PyTorch 模块上自定义方法
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'
    
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 通过实例化上面定义的类来构造我们的模型
model = DynamicNet()

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

1999 3313.55224609375
3999 1525.671630859375
5999 700.5814208984375
7999 346.0111389160156
9999 158.0843963623047
11999 78.30085754394531
13999 41.366275787353516
15999 24.08107566833496
17999 15.956063270568848
19999 12.25631332397461
21999 10.454581260681152
23999 9.577637672424316
25999 8.980056762695312
27999 9.027199745178223
29999 8.65000057220459
Result: y = 0.009211992844939232 + 0.853203535079956 x + -0.002156247617676854 x^2 + -0.09316753596067429 x^3 + 0.00012350086763035506 x^4 ? + 0.00012350086763035506 x^5 ?
